In [1]:
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

In [2]:
df = pd.read_csv('seoul_filtered_cafe_중복제거.csv')

In [3]:
df.head()

,업소명,행정동명,소재지도로명,소재지지번,영업자시작일,소재지시작일,폐업일자,업소위치,총면적,카페명,최신시작일
0,새샘,인헌동,,서울특별시 관악구 봉천동 1639번지 10호,2002-05-31,2001-08-22,2005-06-02,지하,147.76,새샘,NaN
1,왕,신사동,,서울특별시 관악구 신림동 580번지 2호,2001-04-27,2001-04-27,2002-05-20,NaN,175.18,왕,NaN
2,원다방,청룡동,,서울특별시 관악구 봉천동 862번지 9호,2001-04-04,2001-04-04,2002-04-01,NaN,159.58,원다방,NaN
3,!(느낌),서교동,"서울특별시 마포구 동교로17안길 10, (서교동,1층)",서울특별시 마포구 서교동 461번지 2호 1층,2010-12-10,2010-12-10,2017-02-07,지상,96.70,!(느낌),NaN
4,# (샵),서교동,"서울특별시 마포구 독막로7길 45, (서교동, 3층)",서울특별시 마포구 서교동 395번지 23호,2013-02-14,2012-01-16,2014-01-28,지상,57.12,# (샵),NaN


In [27]:
len(df[df['소재지도로명'] == '    '])

19537

In [5]:
for i in range(len(df)):
    if df.iloc[i,2] == '    ':
        df.iloc[i,2] = None

In [6]:
df['소재지도로명'].isna().sum()

np.int64(19537)

소재지도로명이 빈칸으로 되어있는 행을 None값으로 바꿔줌.

In [4]:
import requests

# 도로명주소 변환 API를 호출하는 함수
def convert_address(jibun_address):
    url = f"http://www.juso.go.kr/addrlink/addrLinkApi.do"
    params = {
        'confmKey': 'devU01TX0FVVEgyMDI0MTAxMjIwMTcxMjExNTE1MDc=',
        'currentPage': 1,
        'countPerPage': 10,
        'keyword': jibun_address,
        'resultType': 'json'
    }
    response = requests.get(url, params=params)
    data = response.json()
    
    # 결과 출력
    if data['results']['common']['totalCount'] > '0':
        road_address = data['results']['juso'][0]['roadAddr']
        return road_address
    else:
        return None


In [7]:
# 정규표현식 사용해서 '번지' 기준으로 첫 번째 파트 추출
df['소재지지번_주소'] = df['소재지지번'].str.extract(r'^(.*?번지\s?\d*호?)')

# 나머지 부분 추출 (번지, 호 이후에 나오는 내용)
df['소재지지번_상세주소'] = df['소재지지번'].str.replace(r'^.*?번지\s?\d*호?\s?', '', regex=True)



In [8]:
df['소재지지번_주소']

0        서울특별시 관악구 봉천동  1639번지 10호
1          서울특별시 관악구 신림동  580번지 2호
2          서울특별시 관악구 봉천동  862번지 9호
3          서울특별시 마포구 서교동  461번지 2호
4         서울특별시 마포구 서교동  395번지 23호
                   ...            
70497      서울특별시 서초구 방배동  863번지 4호
70498       서울특별시 마포구 신공덕동  171번지 
70499        서울특별시 강서구 등촌동  849번지 
70500        서울특별시 송파구 삼전동  3번지 4호
70501     서울특별시 중랑구 망우동  147번지 21호
Name: 소재지지번_주소, Length: 70502, dtype: object

In [ ]:
address_0 = []
for i in range(0,23658):
    if df.loc[i]['소재지지번_주소'] == None:
        address_0.append(None)
    else:
        address_0.append(convert_address(df.loc[i]['소재지지번_주소']))

In [102]:
len(address_0)

20349

In [104]:
address_0_df = pd.DataFrame(address_0, columns=['도로명주소'])
address_0_df.to_csv('filtered_cafe_1010_address_0.csv')

In [9]:
address_0_df = pd.read_csv('filtered_cafe_1010_address_0.csv', index_col=['Unnamed: 0'])

In [14]:
address_20349 = []
for i in range(20349,23658):
    if df.loc[i]['소재지지번_주소'] == None:
        address_20349.append(None)
    else:
        address_20349.append(convert_address(df.loc[i]['소재지지번_주소']))

In [15]:
len(address_20349)

3309

In [17]:
address_20349_df = pd.DataFrame(address_20349, columns=['도로명주소'])
address_20349_df.to_csv('filtered_cafe_1010_address_20349.csv')

In [70]:
address = []
for i in range(23658, len(df)):
    if df.loc[i]['소재지지번_주소'] == None:
        address.append(None)
    else:
        address.append(convert_address(df.loc[i]['소재지지번_주소']))

In [81]:
len(address)

46844

In [79]:
address_df = pd.DataFrame(address, columns=['도로명주소'])

In [93]:
address_df.to_csv('filtered_cafe_1010_address_after23658.csv', encoding='cp949')

In [19]:
address_23658_df = pd.read_csv('filtered_cafe_1010_address_after23658.csv', encoding='cp949', index_col='Unnamed: 0')

In [28]:
address_70502_df = pd.concat([address_0_df, address_20349_df, address_23658_df], axis=0)
address_70502_df = address_70502_df.reset_index()

In [30]:
address_70502_df.drop('index', axis=1, inplace=True)

In [36]:
address_70502_df

,도로명주소
0,서울특별시 관악구 인헌6길 5-8 (봉천동)
1,NaN
2,서울특별시 관악구 관악로 171 (봉천동)
3,서울특별시 마포구 동교로 129 (서교동)
4,서울특별시 마포구 독막로3길 17 (서교동)
...,...
70497,서울특별시 서초구 도구로 123-2 (방배동)
70498,"서울특별시 마포구 백범로 202 (신공덕동, 마포케이씨씨웰츠타워)"
70499,서울특별시 강서구 양천로 400-12 (등촌동)
70500,서울특별시 송파구 삼전로 92 (삼전동)


In [24]:
print(len(address_70502_df))
print(len(df))

70502
70502


In [33]:
for i in range(len(df)):
    if df.iloc[i,2] == None:
         df.iloc[i,2] = address_70502_df.iloc[i,0]
    else:
         df.iloc[i,2] == None

In [34]:
df['소재지도로명'].isna().sum()

np.int64(1408)

도로명주소 결측치 19537 -> 1408

In [35]:
df[df['소재지도로명'].isna()]

,업소명,행정동명,소재지도로명,소재지지번,영업자시작일,소재지시작일,폐업일자,업소위치,총면적,카페명,최신시작일,소재지지번_주소,소재지지번_상세주소
1,왕,신사동,NaN,서울특별시 관악구 신림동 580번지 2호,2001-04-27,2001-04-27,2002-05-20,NaN,175.18,왕,NaN,서울특별시 관악구 신림동 580번지 2호,
149,(주)더박스일원역점 더박스 라운지,개포2동,NaN,서울특별시 강남구 일원동 214번지 일원역구내348-10,2011-04-25,2011-04-25,NaN,지하,45.20,(주)더박스일원역점 더박스 라운지,2011-04-25,서울특별시 강남구 일원동 214번지,일원역구내348-10
173,(주)동서헤르젠무교점,명동,NaN,서울특별시 중구 다동 183번지 0호,1997-04-07,1997-04-07,1997-04-07,NaN,210.62,(주)동서헤르젠무교점,NaN,서울특별시 중구 다동 183번지 0호,
180,(주)동원홈푸드 카페키니 연세대점,신촌동,NaN,서울특별시 서대문구 대신동 100번지 국제2학사,2012-12-20,2010-02-25,2013-01-08,지하,910.80,(주)동원홈푸드 카페키니 연세대점,NaN,서울특별시 서대문구 대신동 100번지,국제2학사
659,(주)종로크라운,종로1.2.3.4가동,NaN,"서울특별시 종로구 낙원동 25, 33, 35번지 지상2층",2002-05-20,2002-05-20,NaN,지상,108.51,(주)종로크라운,2002-05-20,"서울특별시 종로구 낙원동 25, 33, 35번지",지상2층
...,...,...,...,...,...,...,...,...,...,...,...,...,...
70235,희,영등포동,NaN,서울특별시 영등포구 영등포동2가 13번지 4호,1993-02-06,1993-02-06,1993-10-19,NaN,165.96,희,NaN,서울특별시 영등포구 영등포동2가 13번지 4호,
70250,희,중림동,NaN,서울특별시 중구 만리동2가 295번지 3호,1991-12-12,1991-12-12,1991-10-08,NaN,131.60,희,NaN,서울특별시 중구 만리동2가 295번지 3호,
70288,희당,서초제4동,NaN,서울특별시 서초구 서초동 1073번지 6호,1998-12-19,1998-12-19,1991-12-31,NaN,195.45,희당,NaN,서울특별시 서초구 서초동 1073번지 6호,
70294,희망,개포1동,NaN,서울특별시 강남구 개포동 0번지,1983-08-26,1983-08-26,1995-05-24,NaN,128.68,희망,NaN,서울특별시 강남구 개포동 0번지,


In [ ]:
import re

# 여러 층 포함 -> 복층 / 한 층만 -> 단층
def is_multifloor(row):
  
# 열 합치고
    address = str(row['소재지도로명']) + ' ' + str(row['소재지지번'])
    
    # 단층 패턴 
    single_floor_patterns = [r'\d+층(?!,)', r'지하\d+층(?!,)', r'B\d+층(?!,)'] 
    
    multi_floor_patterns = [
        r'\d+층~\d+층',        #'1~2층'
        r'지하\d+층~\d+층',    #'지하1층~1층'
        r'B\d+층~\d+층',       #'B1층~1층'
        r'\d+층, \d+층',       #'1층, 2층'
        r'\d+층, 지하\d+층',   #'1층, 지하1층'
        r'지하\d+층, \d+층',   #'지하1층, 1층'
        r'\d+,\d+층',          #'1,2층'
        r'\d+~\d+층',
        r'지하\d+층,\d+층'           #'1~2층'
    ]
    
    # 복층 패턴이 있는지 확인
    if any(re.search(pattern, address) for pattern in multi_floor_patterns):
        return 1  # 복층
    # 단층 패턴이 있는지 확인
    elif any(re.search(pattern, address) for pattern in single_floor_patterns):
        return 0  # 단층
    else:
        return 0 

# 파생변수 만드러
df['복층여부'] = df.apply(is_multifloor, axis=1)





In [ ]:
df[['업소명', '소재지도로명', '소재지지번', '복층여부']].head()

In [40]:
df[df['복층여부']==0][['업소명', '소재지도로명', '소재지지번', '복층여부']].head()

,업소명,소재지도로명,소재지지번,복층여부
0,새샘,서울특별시 관악구 인헌6길 5-8 (봉천동),서울특별시 관악구 봉천동 1639번지 10호,0
1,왕,NaN,서울특별시 관악구 신림동 580번지 2호,0
2,원다방,서울특별시 관악구 관악로 171 (봉천동),서울특별시 관악구 봉천동 862번지 9호,0
3,!(느낌),"서울특별시 마포구 동교로17안길 10, (서교동,1층)",서울특별시 마포구 서교동 461번지 2호 1층,0
4,# (샵),"서울특별시 마포구 독막로7길 45, (서교동, 3층)",서울특별시 마포구 서교동 395번지 23호,0


In [41]:
df[df['복층여부']==1][['업소명', '소재지도로명', '소재지지번', '복층여부']].head()

,업소명,소재지도로명,소재지지번,복층여부
12,#dono,"서울특별시 용산구 두텁바위로 81-2, 1,2층 (후암동)",서울특별시 용산구 후암동 262번지 2호,1
31,(The coffee) 더 커피,"서울특별시 종로구 종로40가길 4, 지상1,2층 (종로5가)","서울특별시 종로구 종로5가 279번지 2호 지상1,2층",1
65,(주) 할리스에프앤비신촌점,"서울특별시 서대문구 연세로 34, 1~3층 (창천동)","서울특별시 서대문구 창천동 31번지 12호 ,13 (1,2,3층)",1
105,(주)공항앤드테라스,"서울특별시 강서구 하늘길 38, 김포공항, 롯데몰, 스카이시티 1,6층 (방화동)","서울특별시 강서구 방화동 886번지 김포공항, 롯데몰, 스카이시티 1,6층",1
108,(주)굿브레드 컴퍼니,"서울특별시 중구 을지로 6, (을지로1가,재능빌딩 (1,2층))","서울특별시 중구 을지로1가 192번지 11호 재능빌딩 (1,2층)",1


In [42]:
df.to_csv('backup_1013.csv', index=False)

## 좌표 변환

In [2]:
import pandas as pd
df = pd.read_csv('backup_1013.csv')

In [3]:
df.head()

,업소명,행정동명,소재지도로명,소재지지번,영업자시작일,소재지시작일,폐업일자,업소위치,총면적,카페명,최신시작일,소재지지번_주소,소재지지번_상세주소,복층여부
0,새샘,인헌동,서울특별시 관악구 인헌6길 5-8 (봉천동),서울특별시 관악구 봉천동 1639번지 10호,2002-05-31,2001-08-22,2005-06-02,지하,147.76,새샘,NaN,서울특별시 관악구 봉천동 1639번지 10호,,0
1,왕,신사동,NaN,서울특별시 관악구 신림동 580번지 2호,2001-04-27,2001-04-27,2002-05-20,NaN,175.18,왕,NaN,서울특별시 관악구 신림동 580번지 2호,,0
2,원다방,청룡동,서울특별시 관악구 관악로 171 (봉천동),서울특별시 관악구 봉천동 862번지 9호,2001-04-04,2001-04-04,2002-04-01,NaN,159.58,원다방,NaN,서울특별시 관악구 봉천동 862번지 9호,,0
3,!(느낌),서교동,"서울특별시 마포구 동교로17안길 10, (서교동,1층)",서울특별시 마포구 서교동 461번지 2호 1층,2010-12-10,2010-12-10,2017-02-07,지상,96.70,!(느낌),NaN,서울특별시 마포구 서교동 461번지 2호,1층,0
4,# (샵),서교동,"서울특별시 마포구 독막로7길 45, (서교동, 3층)",서울특별시 마포구 서교동 395번지 23호,2013-02-14,2012-01-16,2014-01-28,지상,57.12,# (샵),NaN,서울특별시 마포구 서교동 395번지 23호,,0


In [18]:
import requests
import pandas as pd

# 네이버 API 클라이언트 정보
client_id = "3fmd7xia0h"
client_secret = "uy5gOT87pjBgIeAmXCH2dpXVBKK090HceaprqeKL"

# API 요청 함수 정의
def get_coordinates(address):
    endpoint = "https://naveropenapi.apigw.ntruss.com/map-geocode/v2/geocode"
    url = f"{endpoint}?query={address}"

    # API 요청 헤더
    headers = {
        "X-NCP-APIGW-API-KEY-ID": client_id,
        "X-NCP-APIGW-API-KEY": client_secret,
    }

    # API 호출 및 좌표 추출
    res = requests.get(url, headers=headers)
    if res.status_code == 200:
        data = res.json()
        if data['status'] == 'OK' and data['meta']['totalCount'] > 0:
            x = data['addresses'][0]['x']
            y = data['addresses'][0]['y']
            return pd.Series([x, y])
    return pd.Series([None, None])

# 주소 열에 대해 좌표를 적용하여 x, y 좌표 열 추가
df[['x', 'y']] = df['소재지도로명'].apply(get_coordinates)

# 결과 확인
df.head()


NameError: name 'df' is not defined

In [5]:
df[df['x'].isna()]

,업소명,행정동명,소재지도로명,소재지지번,영업자시작일,소재지시작일,폐업일자,업소위치,총면적,카페명,최신시작일,소재지지번_주소,소재지지번_상세주소,복층여부,x,y
1,왕,신사동,NaN,서울특별시 관악구 신림동 580번지 2호,2001-04-27,2001-04-27,2002-05-20,NaN,175.18,왕,NaN,서울특별시 관악구 신림동 580번지 2호,,0,None,None
83,(주)공차코리아,화양동,"서울특별시 광진구 능동로 지하 210, 지하1층 726-1705호 (화양동, 7호선...",서울특별시 광진구 화양동 164번지 1호 7호선 어린이대공원역 지하1층 726-...,2014-11-12,2014-11-04,2017-05-11,지하,58.71,(주)공차코리아,NaN,서울특별시 광진구 화양동 164번지 1호,7호선 어린이대공원역 지하1층 726-1705호,0,None,None
121,(주)나이스커피시스템,역삼1동,"서울특별시 강남구 강남대로 지하 396, (역삼동, 강남역 강남스퀘어 (11번~12...",서울특별시 강남구 역삼동 804번지,2017-05-11,2017-05-11,2017-05-16,지상,0.00,(주)나이스커피시스템,NaN,서울특별시 강남구 역삼동 804번지,NaN,0,None,None
149,(주)더박스일원역점 더박스 라운지,개포2동,NaN,서울특별시 강남구 일원동 214번지 일원역구내348-10,2011-04-25,2011-04-25,NaN,지하,45.20,(주)더박스일원역점 더박스 라운지,2011-04-25,서울특별시 강남구 일원동 214번지,일원역구내348-10,0,None,None
173,(주)동서헤르젠무교점,명동,NaN,서울특별시 중구 다동 183번지 0호,1997-04-07,1997-04-07,1997-04-07,NaN,210.62,(주)동서헤르젠무교점,NaN,서울특별시 중구 다동 183번지 0호,,0,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
70288,희당,서초제4동,NaN,서울특별시 서초구 서초동 1073번지 6호,1998-12-19,1998-12-19,1991-12-31,NaN,195.45,희당,NaN,서울특별시 서초구 서초동 1073번지 6호,,0,None,None
70294,희망,개포1동,NaN,서울특별시 강남구 개포동 0번지,1983-08-26,1983-08-26,1995-05-24,NaN,128.68,희망,NaN,서울특별시 강남구 개포동 0번지,NaN,0,None,None
70348,히든폼,안암동,"서울특별시 성북구 인촌로8길 73, 1층 (안암동3가)",서울특별시 성북구 안암동3가 103번지 지상1층,2019-03-08,2019-03-08,NaN,지상,66.94,히든폼,2019-03-08,서울특별시 성북구 안암동3가 103번지,지상1층,0,None,None
70403,히코코,천호제2동,서울특별시 강동구 천호대로 지하997 (천호동),서울특별시 강동구 천호동 455번지 8호 현대백화점 2층,2002-01-02,2002-01-02,2004-02-03,지상,95.70,히코코,NaN,서울특별시 강동구 천호동 455번지 8호,현대백화점 2층,0,None,None


In [7]:
df.drop(['소재지지번_주소','소재지지번_상세주소'], axis=1, inplace=True)

In [6]:
df['영업자시작일'] = pd.to_datetime(df['영업자시작일'], format='%Y-%m-%d')
df['소재지시작일'] = pd.to_datetime(df['소재지시작일'], format='%Y-%m-%d')

In [7]:
df_2010 = df[df['영업자시작일'].dt.year >= 2010]
df_2010.reset_index(inplace=True)
df_2010.drop('index',axis=1,inplace=True)

C:\Users\swc08\AppData\Local\Temp\ipykernel_23364\3696430844.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_2010.drop('index',axis=1,inplace=True)


In [8]:
len(df_2010)

49930

In [9]:
df_2010[df_2010['x'].isna()]

,업소명,행정동명,소재지도로명,소재지지번,영업자시작일,소재지시작일,폐업일자,업소위치,총면적,카페명,최신시작일,복층여부,x,y
72,(주)공차코리아,화양동,"서울특별시 광진구 능동로 지하 210, 지하1층 726-1705호 (화양동, 7호선...",서울특별시 광진구 화양동 164번지 1호 7호선 어린이대공원역 지하1층 726-...,2014-11-12,2014-11-04,2017-05-11,지하,58.71,(주)공차코리아,NaN,0,NaN,NaN
107,(주)나이스커피시스템,역삼1동,"서울특별시 강남구 강남대로 지하 396, (역삼동, 강남역 강남스퀘어 (11번~12...",서울특별시 강남구 역삼동 804번지,2017-05-11,2017-05-11,2017-05-16,지상,0.00,(주)나이스커피시스템,NaN,0,NaN,NaN
132,(주)더박스일원역점 더박스 라운지,개포2동,NaN,서울특별시 강남구 일원동 214번지 일원역구내348-10,2011-04-25,2011-04-25,NaN,지하,45.20,(주)더박스일원역점 더박스 라운지,2011-04-25,0,NaN,NaN
155,(주)동원홈푸드 샌드프레소 이수역점,사당제1동,"서울특별시 동작구 사당로 지하 310, B1층 (사당동, 이수역지하 736-1704)",서울특별시 동작구 사당동 144번지 4호,2014-09-29,2014-09-29,2015-08-27,지하,51.00,(주)동원홈푸드 샌드프레소 이수역점,NaN,0,NaN,NaN
158,(주)동원홈푸드 카페키니 연세대점,신촌동,NaN,서울특별시 서대문구 대신동 100번지 국제2학사,2012-12-20,2010-02-25,2013-01-08,지하,910.80,(주)동원홈푸드 카페키니 연세대점,NaN,0,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
49705,휴앤안,대림제1동,"서울특별시 영등포구 신길로9길 5, (대림동, 1층 일부)",서울특별시 영등포구 대림동 913번지 7호 1층 일부,2013-08-13,2013-08-13,2017-05-19,지상,95.21,휴앤안,NaN,0,NaN,NaN
49747,흙궁,을지로동,"서울특별시 중구 을지로 지하 218, 421-2호 (을지로5가)",서울특별시 중구 을지로5가 35번지 6호,2022-02-03,2022-02-03,2023-12-07,NaN,65.00,흙궁,NaN,0,NaN,NaN
49748,흙다방,명동,"서울특별시 중구 을지로 지하 88, 을특7호 (을지로2가)",서울특별시 중구 을지로2가 149번지 10호,2022-01-11,2022-01-11,NaN,NaN,31.14,흙다방,2022-01-11,0,NaN,NaN
49749,흙바둑카페,을지로동,"서울특별시 중구 을지로 지하 218, 449호 (을지로5가)",서울특별시 중구 을지로5가 35번지 6호,2022-02-15,2022-02-15,NaN,NaN,31.36,흙바둑카페,2022-02-15,0,NaN,NaN


In [10]:
df_2010.to_csv('seoul_filtered_cafe_2010이후.csv', index=False)

In [8]:
import pandas as pd
cafe = pd.read_csv('카페/seoul_filtered_cafe_2010이후.csv')

In [9]:
cafe_na = cafe[cafe['x'].isna()]

In [10]:
cafe_na

,업소명,행정동명,소재지도로명,소재지지번,영업자시작일,소재지시작일,폐업일자,업소위치,총면적,카페명,최신시작일,복층여부,x,y
72,(주)공차코리아,화양동,"서울특별시 광진구 능동로 지하 210, 지하1층 726-1705호 (화양동, 7호선...",서울특별시 광진구 화양동 164번지 1호 7호선 어린이대공원역 지하1층 726-...,2014-11-12,2014-11-04,2017-05-11,지하,58.71,(주)공차코리아,NaN,0,NaN,NaN
107,(주)나이스커피시스템,역삼1동,"서울특별시 강남구 강남대로 지하 396, (역삼동, 강남역 강남스퀘어 (11번~12...",서울특별시 강남구 역삼동 804번지,2017-05-11,2017-05-11,2017-05-16,지상,0.00,(주)나이스커피시스템,NaN,0,NaN,NaN
132,(주)더박스일원역점 더박스 라운지,개포2동,NaN,서울특별시 강남구 일원동 214번지 일원역구내348-10,2011-04-25,2011-04-25,NaN,지하,45.20,(주)더박스일원역점 더박스 라운지,2011-04-25,0,NaN,NaN
155,(주)동원홈푸드 샌드프레소 이수역점,사당제1동,"서울특별시 동작구 사당로 지하 310, B1층 (사당동, 이수역지하 736-1704)",서울특별시 동작구 사당동 144번지 4호,2014-09-29,2014-09-29,2015-08-27,지하,51.00,(주)동원홈푸드 샌드프레소 이수역점,NaN,0,NaN,NaN
158,(주)동원홈푸드 카페키니 연세대점,신촌동,NaN,서울특별시 서대문구 대신동 100번지 국제2학사,2012-12-20,2010-02-25,2013-01-08,지하,910.80,(주)동원홈푸드 카페키니 연세대점,NaN,0,NaN,NaN


In [22]:
cafe[['도로명주소', '상세주소']] = cafe['소재지도로명'].str.split(',', n=1, expand=True)

In [23]:
cafe['도로명주소_지하X'] = cafe['도로명주소'].str.replace('지하', '', regex=False).str.replace(r'\s+', ' ', regex=True).str.strip()

In [24]:
# 결측치가 있는 행에 대해 좌표 변환 함수 적용
cafe.loc[cafe['x'].isna(), ['x', 'y']] = cafe.loc[cafe['x'].isna(), '도로명주소_지하X'].apply(
    lambda addr: pd.Series(get_coordinates(addr))
)

In [25]:
cafe

,업소명,행정동명,소재지도로명,소재지지번,영업자시작일,소재지시작일,폐업일자,업소위치,총면적,카페명,최신시작일,복층여부,x,y,도로명주소,상세주소,도로명주소_지하X
0,!(느낌),서교동,"서울특별시 마포구 동교로17안길 10, (서교동,1층)",서울특별시 마포구 서교동 461번지 2호 1층,2010-12-10,2010-12-10,2017-02-07,지상,96.70,!(느낌),NaN,0,126.916749,37.555387,서울특별시 마포구 동교로17안길 10,"(서교동,1층)",서울특별시 마포구 동교로17안길 10
1,# (샵),서교동,"서울특별시 마포구 독막로7길 45, (서교동, 3층)",서울특별시 마포구 서교동 395번지 23호,2013-02-14,2012-01-16,2014-01-28,지상,57.12,# (샵),NaN,0,126.919585,37.549846,서울특별시 마포구 독막로7길 45,"(서교동, 3층)",서울특별시 마포구 독막로7길 45
2,#1 shot in coffee(원샷),방이1동,"서울특별시 송파구 올림픽로 424, 올림픽공원 올림픽수영장 1층 (방이동)",서울특별시 송파구 방이동 88번지 올림픽공원,2019-04-01,2019-04-01,NaN,지상,101.44,#1 shot in coffee(원샷),2019-04-01,0,127.122959,37.520934,서울특별시 송파구 올림픽로 424,올림픽공원 올림픽수영장 1층 (방이동),서울특별시 송파구 올림픽로 424
3,#113,신림동,"서울특별시 관악구 봉천로12길 39, 1층 113호 (신림동)",서울특별시 관악구 신림동 1426번지 7호,2016-09-12,2015-11-12,NaN,지상,49.50,#113,2016-09-12,0,126.930207,37.486910,서울특별시 관악구 봉천로12길 39,1층 113호 (신림동),서울특별시 관악구 봉천로12길 39
4,#52랩,명동,"서울특별시 중구 퇴계로14길 19, 1층 (회현동3가)",서울특별시 중구 회현동3가 9번지 15호 1층,2016-11-09,2016-11-09,2018-07-20,지상,20.27,#52랩,NaN,0,126.983780,37.559687,서울특별시 중구 퇴계로14길 19,1층 (회현동3가),서울특별시 중구 퇴계로14길 19
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
49925,？커피로스터스(Buff Coffee Roasters),방배제1동,"서울특별시 서초구 도구로 123-2, 삼흥빌딩 1층 (방배동)",서울특별시 서초구 방배동 863번지 4호 삼흥빌딩,2022-09-29,2022-09-29,NaN,지상,0.00,？커피로스터스(Buff Coffee Roasters),2022-09-29,0,126.985825,37.487616,서울특별시 서초구 도구로 123-2,삼흥빌딩 1층 (방배동),서울특별시 서초구 도구로 123-2
49926,？킹베이비,공덕동,"서울특별시 마포구 백범로 202, (신공덕동, 마포KCC웰츠타워 2층 208~209호)",서울특별시 마포구 신공덕동 171번지,2012-08-06,2012-08-06,2019-06-14,지상,139.95,？킹베이비,NaN,0,126.952347,37.543078,서울특별시 마포구 백범로 202,"(신공덕동, 마포KCC웰츠타워 2층 208~209호)",서울특별시 마포구 백범로 202
49927,？？베이크,가양제1동,"서울특별시 강서구 양천로 400-12, 더리브골드타워 1층 117호 (등촌동)",서울특별시 강서구 등촌동 849번지 더리브골드타워,2023-11-21,2023-11-21,NaN,지상,14.28,？？베이크,2023-11-21,0,126.845756,37.565327,서울특별시 강서구 양천로 400-12,더리브골드타워 1층 117호 (등촌동),서울특별시 강서구 양천로 400-12
49928,？？카페,삼전동,"서울특별시 송파구 삼전로 88, 근화빌딩 1층 (삼전동)",서울특별시 송파구 삼전동 3번지 4호 근화빌딩,2021-10-05,2021-10-05,NaN,지상,29.75,？？카페,2021-10-05,0,127.089886,37.506123,서울특별시 송파구 삼전로 88,근화빌딩 1층 (삼전동),서울특별시 송파구 삼전로 88


In [26]:
cafe['x'].isna().sum()

np.int64(588)